# Preprocesamiento del set de datos

Se preparará el set de datos en 2 etapas:

1) Analisis
2) Procesamiento

En la etapa de analisis se pretende determinar que datos son relevantes y cuales no, y determinar que datos textuales pueden ser transformados a datos numericos y como se realizara esta "traduccion". 

En la etapa de procesamiento se realizaran los cambios necesarios al set de datos, asi como tambien la creacion de nuevas columnas a partir de otras.





### Levantamos el set de datos completo

In [1]:
import pandas as pd

dataset = pd.read_csv("Accidente.csv", true_values = ['true','True'], false_values = ['false','False'])


In [2]:
dataset.head()

,ID,Source,TMC,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,MapQuest,201.0,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,MapQuest,201.0,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,...,False,False,False,False,False,False,Night,Night,Night,Day
2,A-3,MapQuest,201.0,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,...,False,False,False,False,True,False,Night,Night,Day,Day
3,A-4,MapQuest,201.0,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,...,False,False,False,False,False,False,Night,Day,Day,Day
4,A-5,MapQuest,201.0,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,...,False,False,False,False,True,False,Day,Day,Day,Day


In [3]:
dataset.dtypes

ID                        object
Source                    object
TMC                      float64
Severity                   int64
Start_Time                object
End_Time                  object
Start_Lat                float64
Start_Lng                float64
End_Lat                  float64
End_Lng                  float64
Distance(mi)             float64
Description               object
Number                   float64
Street                    object
Side                      object
City                      object
County                    object
State                     object
Zipcode                   object
Country                   object
Timezone                  object
Airport_Code              object
Weather_Timestamp         object
Temperature(F)           float64
Wind_Chill(F)            float64
Humidity(%)              float64
Pressure(in)             float64
Visibility(mi)           float64
Wind_Direction            object
Wind_Speed(mph)          float64
Precipitat

In [4]:
dataset = dataset.drop("ID",axis="columns") #No aporta informacion 
dataset = dataset.drop("Source",axis="columns") #No aporta info sobre la severidad
dataset = dataset.drop("TMC",axis="columns") #No aporta mucha informacion sobre la severidad
dataset = dataset.drop("Country",axis="columns") #Hay un solo valor en todas las columnas
dataset = dataset.drop("Description",axis="columns") #Partes de la descripcion se encuentran en otras columnas
dataset = dataset.drop("End_Lat",axis="columns") #No aportan mucha info
dataset = dataset.drop("End_Lng",axis="columns") #No aportan mucha info
dataset = dataset.drop("Number",axis="columns") #Tiene mas valores nulos que medidos
dataset = dataset.drop("Weather_Timestamp",axis="columns") #No coincide con el horario del accidente, por lo que no es relevante
dataset = dataset.drop("Airport_Code",axis="columns") #No aportan mucha info
dataset = dataset.drop("Timezone",axis="columns") #No aportan mucha info
dataset = dataset.drop("Zipcode",axis="columns") #No aportan mucha info
dataset = dataset.drop("State",axis="columns") #No aportan mucha info
dataset = dataset.drop("County",axis="columns") #No aportan mucha info
dataset = dataset.drop("Wind_Chill(F)",axis="columns") #No aportan mucha info
dataset = dataset.drop("Civil_Twilight",axis="columns") #No aportan mucha info
dataset = dataset.drop("Nautical_Twilight",axis="columns") #No aportan mucha info
dataset = dataset.drop("Astronomical_Twilight",axis="columns") #No aportan mucha info

dataset.shape

(2974335, 31)

### Transformamos las columnas categoricas

In [5]:
dataset['Side'] = dataset['Side'].astype('category')
dataset['Wind_Direction'] = dataset['Wind_Direction'].astype('category')
dataset['Weather_Condition'] = dataset['Weather_Condition'].astype('category')
dataset['Sunrise_Sunset'] = dataset['Sunrise_Sunset'].astype('category')

dataset['Temperature(F)'] = dataset['Temperature(F)'].astype('float64')

### Nos fijamos la cantidad de valores nulos que tienen nuestras columnas

In [6]:
dataset.isnull().sum()

Severity                   0
Start_Time                 0
End_Time                   0
Start_Lat                  0
Start_Lng                  0
Distance(mi)               0
Street                     0
Side                       0
City                      83
Temperature(F)         56063
Humidity(%)            59173
Pressure(in)           48142
Visibility(mi)         65691
Wind_Direction         45101
Wind_Speed(mph)       440840
Precipitation(in)    1998358
Weather_Condition      65932
Amenity                    0
Bump                       0
Crossing                   0
Give_Way                   0
Junction                   0
No_Exit                    0
Railway                    0
Roundabout                 0
Station                    0
Stop                       0
Traffic_Calming            0
Traffic_Signal             0
Turning_Loop               0
Sunrise_Sunset            93
dtype: int64

In [7]:
#dataset["Precipitation(in)"] = dataset["Precipitation(in)"].fillna(0)
#dataset["Wind_Speed(mph)"] = dataset["Wind_Speed(mph)"].fillna(0)

#dataset = dataset[dataset['Weather_Condition'].isnull() != True]
#dataset = dataset[dataset['Humidity(%)'].isnull() != True]
#dataset = dataset[dataset['Visibility(mi)'].isnull() != True]
#dataset = dataset[dataset['Visibility(mi)'].isnull() != True]
dataset = dataset.dropna()

In [8]:
dataset.isnull().sum()

Severity             0
Start_Time           0
End_Time             0
Start_Lat            0
Start_Lng            0
Distance(mi)         0
Street               0
Side                 0
City                 0
Temperature(F)       0
Humidity(%)          0
Pressure(in)         0
Visibility(mi)       0
Wind_Direction       0
Wind_Speed(mph)      0
Precipitation(in)    0
Weather_Condition    0
Amenity              0
Bump                 0
Crossing             0
Give_Way             0
Junction             0
No_Exit              0
Railway              0
Roundabout           0
Station              0
Stop                 0
Traffic_Calming      0
Traffic_Signal       0
Turning_Loop         0
Sunrise_Sunset       0
dtype: int64

### Time_Elapsed

La hora de inicio y la de fin por si solas no son tan relevantes para determinar la severidad, pero la duracion del intervalo
podria considerarse un factor para determinar la misma. Un accidente que demora mucho tiempo en considerarse "terminado" podria considerarse como muy severo

In [9]:
dataset['Start_Time'] = dataset['Start_Time'].astype('datetime64')
dataset['End_Time'] = dataset['End_Time'].astype('datetime64')
dataset.dtypes

Severity                      int64
Start_Time           datetime64[ns]
End_Time             datetime64[ns]
Start_Lat                   float64
Start_Lng                   float64
Distance(mi)                float64
Street                       object
Side                       category
City                         object
Temperature(F)              float64
Humidity(%)                 float64
Pressure(in)                float64
Visibility(mi)              float64
Wind_Direction             category
Wind_Speed(mph)             float64
Precipitation(in)           float64
Weather_Condition          category
Amenity                        bool
Bump                           bool
Crossing                       bool
Give_Way                       bool
Junction                       bool
No_Exit                        bool
Railway                        bool
Roundabout                     bool
Station                        bool
Stop                           bool
Traffic_Calming             

In [10]:
time_elapsed = dataset["End_Time"] - dataset["Start_Time"]#Nos sirve mas la duracion total que el momento de inicio y el de fin

In [11]:
dataset["Time_Elapsed"] = time_elapsed #Entonces creamos una nueva columna

In [12]:
dataset = dataset.drop("Start_Time",axis="columns") #Ya guardamos la duracion del accidente
dataset = dataset.drop("End_Time",axis="columns") #Ya guardamos la duracion del accidente

### Explicacion de la eliminacion de ciertas columnas

La logica por detras de la eleccion es la siguiente: los accidentes de transito son un fenomeno local. Los factores que suelen influenciar mas la severidad de un accidente suelen ser el estado del clima, si la via donde ocurren esta bien señalizada, etc.. 

Como fenomeno local queremos implicar que los factores que influyen sobre el accidente deben estar relativamente cercanos al
mismo, como por ejemplo un cruce sin semaforo, un cartel publicitario que distrae, asfalto mojado/congelado, etc.. Estos son fenomenos que se pueden observar a ojo desde el epicentro del accidente por lo que informacion sobre el codigo de aeropuerto mas cercano o el departamento del estado donde ocurrio el accidente no es relevante a la hora de determinar la severidad.

Un factor que si podemos correlacionar seria la calle, la altura e inclusive la ciudad. Una metropolis suele tener mas accidentes debido a la cantidad de autos que circulan o una calle a determinada altura tiene un pozo que hace que ocurran mas accidentes de lo normal. 

In [13]:
dataset.dtypes

Severity                       int64
Start_Lat                    float64
Start_Lng                    float64
Distance(mi)                 float64
Street                        object
Side                        category
City                          object
Temperature(F)               float64
Humidity(%)                  float64
Pressure(in)                 float64
Visibility(mi)               float64
Wind_Direction              category
Wind_Speed(mph)              float64
Precipitation(in)            float64
Weather_Condition           category
Amenity                         bool
Bump                            bool
Crossing                        bool
Give_Way                        bool
Junction                        bool
No_Exit                         bool
Railway                         bool
Roundabout                      bool
Station                         bool
Stop                            bool
Traffic_Calming                 bool
Traffic_Signal                  bool
T

In [14]:
obj_cols = dataset.select_dtypes(['object']).columns

In [15]:
from sklearn.preprocessing import LabelEncoder

# define ordinal encoding
street_encoder = LabelEncoder()
# transform data
enc_street = street_encoder.fit_transform(dataset["Street"])
dataset["Street"] = enc_street

In [16]:
# define ordinal encoding
city_encoder = LabelEncoder()
# transform data
enc_city = city_encoder.fit_transform(dataset["City"])
dataset["City"] = enc_city

In [17]:
dataset.dtypes

Severity                       int64
Start_Lat                    float64
Start_Lng                    float64
Distance(mi)                 float64
Street                         int64
Side                        category
City                           int64
Temperature(F)               float64
Humidity(%)                  float64
Pressure(in)                 float64
Visibility(mi)               float64
Wind_Direction              category
Wind_Speed(mph)              float64
Precipitation(in)            float64
Weather_Condition           category
Amenity                         bool
Bump                            bool
Crossing                        bool
Give_Way                        bool
Junction                        bool
No_Exit                         bool
Railway                         bool
Roundabout                      bool
Station                         bool
Stop                            bool
Traffic_Calming                 bool
Traffic_Signal                  bool
T

In [18]:
cat_columns = dataset.select_dtypes(['category']).columns
cat_columns

Index(['Side', 'Wind_Direction', 'Weather_Condition', 'Sunrise_Sunset'], dtype='object')

In [19]:
#Aca transformamos los valores de las columnas de datos categoricos a su representacion numerica

dataset = pd.get_dummies(dataset, columns=cat_columns) #get_dummies utiliza one hot encoding en las columnas categoricas

In [20]:
dataset.shape

(933236, 175)

In [21]:
#%store dataset

In [22]:
dataset.dropna()

,Severity,Start_Lat,Start_Lng,Distance(mi),Street,City,Temperature(F),Humidity(%),Pressure(in),Visibility(mi),...,Weather_Condition_Thunderstorms and Rain,Weather_Condition_Thunderstorms and Snow,Weather_Condition_Tornado,Weather_Condition_Volcanic Ash,Weather_Condition_Widespread Dust,Weather_Condition_Widespread Dust / Windy,Weather_Condition_Wintry Mix,Weather_Condition_Wintry Mix / Windy,Sunrise_Sunset_Day,Sunrise_Sunset_Night
5,3,40.100590,-82.925194,0.010,86519,8903,37.9,97.0,29.63,7.0,...,0,0,0,0,0,0,0,0,1,0
9,3,40.100590,-82.925194,0.010,86519,8903,37.4,100.0,29.62,3.0,...,0,0,0,0,0,0,0,0,1,0
11,3,39.932709,-82.830910,0.010,43872,6873,37.4,100.0,29.62,3.0,...,0,0,0,0,0,0,0,0,1,0
14,2,39.972038,-82.913521,0.010,31407,1668,37.4,100.0,29.62,3.0,...,0,0,0,0,0,0,0,0,1,0
20,2,40.052509,-82.882332,0.000,22111,1668,33.8,100.0,29.62,2.0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2974330,2,34.002480,-117.379360,0.543,64004,6959,86.0,40.0,28.92,10.0,...,0,0,0,0,0,0,0,0,1,0
2974331,2,32.766960,-117.148060,0.338,43908,7270,70.0,73.0,29.39,10.0,...,0,0,0,0,0,0,0,0,1,0
2974332,2,33.775450,-117.847790,0.561,37876,6096,73.0,64.0,29.74,10.0,...,0,0,0,0,0,0,0,0,1,0
2974333,2,33.992460,-118.403020,0.772,72931,1904,71.0,81.0,29.62,10.0,...,0,0,0,0,0,0,0,0,1,0


In [27]:
dataset = dataset.reset_index()
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 933236 entries, 0 to 933235
Columns: 176 entries, index to Sunrise_Sunset_Night
dtypes: bool(13), float64(9), int64(4), timedelta64[ns](1), uint8(149)
memory usage: 243.9 MB


In [29]:
dataset.to_feather("dataframe")